### LAYER 1: CONFIGURATION & SETUP

In [1]:
import re
import string
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
import joblib
from collections import defaultdict, Counter
from tqdm.auto import tqdm

import nltk
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Setup stopwords
nltk_stopword = stopwords.words('indonesian')
stopword_id_factory = StopWordRemoverFactory()
sastrawi_stopword = stopword_id_factory.get_stop_words()

additional_stopwords = [
    "nya","sih","mah","r","n","kalo","tuh","ah","b","l","deh","kah","oh","ih","dih","bro","cuy",
    "sa","ya","ok","heh","lo","lu","i","ii","ti","ki","bal","t","al","qur","je","ta","oy","li",
    "h","ar","p","as","hi","v","nge","wkwkwk","dll","nih","ku","a","iii","si","lho","gua","gue",
    "gu","ay","et","opo","ai","un","lol","sus","es","ut","iki","zu","ane","ab","mil","wie","ev",
    "f","kd","st","kar","to","ipo","och","einie","sek","lee","eriii","vii","ile","mi","sel","weh",
    "sb","ra","iin","ske","sur","um","xde","iku","bla","hai","xl","des","duh","we","cc","ag","wan",
    "po","nin","yth","ipu","auch","wes","yaudah","tir","wkwk","wk","mu","les","kor","ppp","au",
    "und","sia","gp","ist","ye","im","ha","hebas","at","pe","bua","qs","bo","ich","they","if",
    "etc","tg","too","als","ngopo","gi","up","ora","ve","kok","go","bv","oi","nom","tr","ui",
    "ana","aku","ahy","kat","tri","iya","tau","kau","bal","an","dah","loh","mbak","e","mak",
    "asa","ayo","ph","vs","wa","xa","jaku","xe","xf","rp","su","ibl","woi","nak","pn","guys",
    "vub","x","aji","my","you","the","this","is","and","of","your","victim","life","why",
    "what","one","no","ber","dm","hehe","he","all","but","okay","just","download","had",
    "hahahaha","niin","walaun","try","xb","ygy","bi","ei","hah","noh","kapai","oke","min",
    "sop","dek","ala","plis","rai","gwe","en","zul","ooo","aing","its","wae","gws","test",
    "bas","by","didu","true","kna","ho","atuh","az","pm","bot","akan","pis","acc","idk",
    "sape","kwa","mohon","minta","ybs","tolong","segera","lanjut","baik" 
]

STOPWORDS = list(set(nltk_stopword + sastrawi_stopword + additional_stopwords))

# Setup stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Configuration
CONFIG = {
    'data_file': 'Data Olah.csv',
    'min_df': 3,
    'max_df': 0.95,
    'top_n_tags': 8,
    'top_k_output': 10,
    'frequency_weight': 0.7,
    'relative_weight': 0.3
}

### LAYER 2: DATA LOADING & COLUMN DETECTION

In [2]:
def find_col(df, candidates):
    """Deteksi kolom berdasarkan daftar kandidat nama"""
    for cand in candidates:
        for c in df.columns:
            if cand.lower() == c.lower().strip():
                return c
    # Fuzzy search
    for cand in candidates:
        for c in df.columns:
            if cand.lower() in c.lower():
                return c
    return None


def load_and_detect_columns(filepath):
    """Load data dan deteksi kolom yang diperlukan"""
    df = pd.read_csv(filepath)
    print(f"Dataset loaded: {len(df)} rows")
    
    columns = {
        'summary': find_col(df, ['Summary','summary','Summary_x']),
        'judul': find_col(df, ['Judul Request_x','Judul_Request_x','Judul Request x','judul request_x','judul']),
        'description': find_col(df, ['Description','Deskripsi','description','deskripsi']),
        'status_x': find_col(df, ['Status_x','Status x','status_x','status']),
        'status_y': find_col(df, ['Status_y','Status y','status_y']),
        'engineer': find_col(df, ['Engineer','engineer','Assignee','assignee','petugas','pegawai']),
        'request_name': find_col(df, ['Request name','Request_name','request_name','request name'])
    }
    
    print("\n=== Detected Columns ===")
    for key, val in columns.items():
        print(f"{key}: {val}")
    
    if columns['engineer'] is None:
        raise ValueError("Kolom Engineer tidak terdeteksi! Set kolom engineer secara manual.")
    
    return df, columns


### LAYER 3: TEXT PREPROCESSING

In [3]:
def cleaning_text(text):
    """Membersihkan teks dari username, URL, simbol, dan angka"""
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub("@[A-Za-z0-9_]+", "", text)
    text = re.sub(r'''(?i)\b((?:https|http?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))\)|[^\s`!()\[\]{};:'".,<>?«»""'']))''', "", text)
    text = re.sub("#[A-Za-z0-9_]+", "", text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = text.split()
    
    # Filter kata dengan 3 huruf berurutan tanpa konsonan
    filtered_tokens = []
    for token in tokens:
        if len(token) > 2:
            has_three_vowels = False
            for i in range(len(token) - 2):
                substring = token[i:i+3]
                if all(char in 'aiueo' for char in substring.lower()):
                    has_three_vowels = True
                    break
            if not has_three_vowels:
                filtered_tokens.append(token)
    
    return " ".join(filtered_tokens)


def casefolding(text):
    """Konversi teks ke lowercase"""
    return text.lower()


def stopwords_removal(text):
    """Menghapus stopwords dari teks"""
    words = text.split()
    filtered_words = [word for word in words if word not in STOPWORDS]
    return ' '.join(filtered_words)


def stemming(text):
    """Melakukan stemming pada teks"""
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)


def tokenizing(text):
    """Tokenisasi teks"""
    return nltk.tokenize.word_tokenize(text)


def preprocess_dataframe(df, columns):
    """Pipeline preprocessing lengkap untuk DataFrame"""
    print("\n=== Preprocessing Data ===")
    
    # Gabungkan kolom untuk text_for_skill
    df['summary_clean'] = df[columns['summary']].fillna("") if columns['summary'] else ""
    df['judul_clean'] = df[columns['judul']].fillna("") if columns['judul'] else ""
    df['description_clean'] = df[columns['description']].fillna("") if columns['description'] else ""
    
    df['text_for_skill'] = (df['summary_clean'].astype(str) + " " + 
                            df['judul_clean'].astype(str) + " " + 
                            df['description_clean'].astype(str)).str.strip()
    
    # Pipeline preprocessing
    print("Applying text cleaning pipeline...")
    df['processed_text'] = df['text_for_skill'].apply(cleaning_text)
    df['processed_text'] = df['processed_text'].apply(casefolding)
    df['processed_text'] = df['processed_text'].apply(stopwords_removal)
    df['processed_text'] = df['processed_text'].apply(stemming)
    df['processed_tokens'] = df['processed_text'].apply(tokenizing)
    df['text_join'] = df['processed_tokens'].apply(
        lambda toks: " ".join(toks) if isinstance(toks, (list, tuple)) else str(toks)
    )
    
    return df

### LAYER 4: DATA FILTERING

In [4]:
def norm_status(v):
    """Normalisasi status"""
    if pd.isna(v):
        return None
    return str(v).strip()


def include_for_skill(row, col_engineer):
    """Tentukan apakah row diikutkan dalam skill profiling"""
    sx = (row['status_x_norm'] or "").lower() if row['status_x_norm'] is not None else None
    sy = (row['status_y_norm'] or "").lower() if row['status_y_norm'] is not None else None
    
    # Exclude cancelled/rejected
    if sx == 'cancelled' and sy in {'cancelled','rejected'}:
        return False
    if sy in {'cancelled','rejected'}:
        return False
    
    # Include jika status handled dan ada engineer + text
    if sx in {'resolved','closed','in progress','need follow up','submitted','waiting for 3rd party','escalated'}:
        if pd.notna(row.get(col_engineer)) and row['text_join'].strip() != '':
            return True
    
    if sy in {'completed','assigned','accepted','waiting'}:
        if pd.notna(row.get(col_engineer)) and row['text_join'].strip() != '':
            return True
    
    # Fallback: engineer exists dan text exists
    if pd.notna(row.get(col_engineer)) and row['text_join'].strip() != '':
        return True
    
    return False


def filter_skill_data(df, columns):
    """Filter data untuk skill profiling"""
    df['status_x_norm'] = df[columns['status_x']].apply(norm_status) if columns['status_x'] else None
    df['status_y_norm'] = df[columns['status_y']].apply(norm_status) if columns['status_y'] else None
    
    df['include_skill'] = df.apply(lambda row: include_for_skill(row, columns['engineer']), axis=1)
    print(f"Rows included for skill profiling: {df['include_skill'].sum()} / {len(df)}")
    
    df_skill = df[df['include_skill']].copy().reset_index(drop=True)
    print(f"Final skill dataset: {len(df_skill)} rows")
    
    return df_skill

### LAYER 5: FEATURE EXTRACTION (TF-IDF)

In [5]:
def extract_tfidf_features(df_skill, config):
    """Ekstraksi fitur TF-IDF dari text"""
    print("\n=== TF-IDF Vectorization ===")
    
    tfidf_tag = TfidfVectorizer(min_df=config['min_df'], max_df=config['max_df'])
    texts = df_skill['text_join'].fillna("").tolist()
    X_tfidf = tfidf_tag.fit_transform(texts)
    terms = tfidf_tag.get_feature_names_out()
    
    print(f"TF-IDF vocab size: {len(terms)}")
    
    return X_tfidf, tfidf_tag, terms


def top_n_terms_from_vector(vec, terms, n=8):
    """Ekstrak top N terms dari TF-IDF vector"""
    arr = vec.toarray().ravel()
    if arr.sum() == 0:
        return []
    idx = np.argsort(arr)[-n:][::-1]
    return [terms[i] for i in idx if arr[i] > 0]


def extract_tags_from_tfidf(X_tfidf, terms, topn=8):
    """Ekstrak top tags dari setiap dokumen"""
    print("Extracting top tags per ticket...")
    top_tags_list = []
    for i in tqdm(range(X_tfidf.shape[0]), desc="Extracting tags"):
        top_tags_list.append(top_n_terms_from_vector(X_tfidf[i], terms, n=topn))
    return top_tags_list

### LAYER 6: SKILL PROFILING

In [6]:
def build_engineer_skill_profiles(df_skill, col_engineer, config):
    """Build skill profiles untuk setiap engineer dengan scoring"""
    print("\n=== Building Engineer Skill Profiles with Scores ===")
    
    # Hitung frekuensi tag per engineer
    eng_tag_counts = defaultdict(Counter)
    for _, row in df_skill.iterrows():
        eng = row[col_engineer]
        if pd.isna(eng) or str(eng).strip() == "":
            continue
        tags = row['extracted_tags'] or []
        eng_tag_counts[eng].update(tags)
    
    # Konversi ke DataFrame dan hitung skor
    rows = []
    for eng, ctr in eng_tag_counts.items():
        total_tickets = sum(ctr.values())
        max_count = max(ctr.values()) if ctr else 1
        
        for tag, cnt in ctr.items():
            # Skor kombinasi: frequency (normalized) + relative importance
            frequency_score = cnt / max_count
            relative_score = cnt / total_tickets
            combined_score = (frequency_score * config['frequency_weight']) + \
                           (relative_score * config['relative_weight'])
            
            rows.append({
                'engineer': eng,
                'tag': tag,
                'count': cnt,
                'score': round(combined_score, 4)
            })
    
    tag_scores_df = pd.DataFrame(rows)
    
    # Build profiles dengan skor
    profiles = {}
    for eng, g in tag_scores_df.groupby('engineer'):
        profiles[eng] = dict(zip(g['tag'], g['score']))
    
    print(f"Engineer profiles created: {len(profiles)} engineers")
    
    return profiles, tag_scores_df


def create_output_dataframe(profiles, topk=10):
    """Buat output DataFrame dengan engineer dan top_tags"""
    engineer_skills_output = []
    for eng, tag_scores in profiles.items():
        # Sort by score descending
        sorted_tags = sorted(tag_scores.items(), key=lambda x: -x[1])[:topk]
        
        # Format: "tag1(score1), tag2(score2), ..."
        top_tags_str = ", ".join([f"{tag}({score:.3f})" for tag, score in sorted_tags])
        
        engineer_skills_output.append({
            'engineer': eng,
            'top_tags': top_tags_str
        })
    
    return pd.DataFrame(engineer_skills_output)

### LAYER 7: CENTROID BUILDING

In [7]:
def build_engineer_centroids(df_skill, X_tfidf, col_engineer):
    """Build centroids untuk setiap engineer"""
    print("\n=== Building Engineer Centroids ===")
    
    engineer_centroids = {}
    for eng, idxs in df_skill.groupby(col_engineer).indices.items():
        rows_tfidf = X_tfidf[list(idxs)]
        centroid = rows_tfidf.mean(axis=0)
        
        if sparse.issparse(centroid):
            centroid = centroid.tocsr()
        elif isinstance(centroid, np.matrix):
            centroid = csr_matrix(np.asarray(centroid))
        elif isinstance(centroid, np.ndarray):
            if centroid.ndim == 1:
                centroid = centroid.reshape(1, -1)
            centroid = csr_matrix(centroid)
        
        engineer_centroids[eng] = centroid
    
    print(f"Engineer centroids created: {len(engineer_centroids)} engineers")
    return engineer_centroids

### LAYER 8: RANKING & RECOMMENDATION

In [8]:
def rank_by_centroid(ticket_text, centroids, tfidf_obj):
    """Ranking engineer berdasarkan centroid similarity"""
    v = tfidf_obj.transform([ticket_text])
    sims = {}
    for eng, cent in centroids.items():
        sim = cosine_similarity(v, cent)[0, 0]
        sims[eng] = float(sim)
    
    # Normalize 0-1
    maxv = max(sims.values()) if sims else 1.0
    if maxv > 0:
        sims = {k: v / maxv for k, v in sims.items()}
    
    return dict(sorted(sims.items(), key=lambda x: -x[1]))


def get_candidates(ticket_text, centroids, tfidf_obj, topk=10, availability=None, workload=None):
    """Get top K engineer candidates dengan filter availability & workload"""
    ranked = rank_by_centroid(ticket_text, centroids, tfidf_obj)
    
    result = []
    for eng, sc in ranked.items():
        if availability is not None:
            if availability.get(eng, 0) == 0:
                continue
        if workload is not None:
            if workload.get(eng, 1) < 0.2:
                continue
        result.append((eng, sc))
        if len(result) >= topk:
            break
    
    return result

### LAYER 9: MODEL PERSISTENCE

In [9]:
def save_models(profiles, engineer_centroids, tfidf_tag):
    """Save semua model ke disk"""
    print("\n=== Saving Models ===")
    
    # Save profiles
    joblib.dump(
        {'tfidf_tag': tfidf_tag, 'profiles': profiles}, 
        'engineer_profiles_tags.joblib'
    )
    print("Saved: engineer_profiles_tags.joblib")
    
    # Save centroids
    joblib.dump(
        {'tfidf_tag': tfidf_tag, 'centroids': engineer_centroids}, 
        'engineer_centroids_tfidf.joblib'
    )
    print("Saved: engineer_centroids_tfidf.joblib")


def save_output(engineer_skills_df):
    """Save output DataFrame"""
    engineer_skills_df.to_csv('engineer_skills_with_scores.csv', index=False)
    print("Saved: engineer_skills_with_scores.csv")

### LAYER 10: MAIN PIPELINE ORCHESTRATION

In [10]:
def main():
    """Main pipeline execution"""
    print("=" * 80)
    print("ENGINEER SKILL PROFILING PIPELINE")
    print("=" * 80)
    
    # Layer 2: Load data
    df, columns = load_and_detect_columns(CONFIG['data_file'])
    
    # Layer 3: Preprocess
    df = preprocess_dataframe(df, columns)
    
    # Layer 4: Filter
    df_skill = filter_skill_data(df, columns)
    
    # Layer 5: Extract TF-IDF
    X_tfidf, tfidf_tag, terms = extract_tfidf_features(df_skill, CONFIG)
    top_tags = extract_tags_from_tfidf(X_tfidf, terms, CONFIG['top_n_tags'])
    df_skill['extracted_tags'] = top_tags
    
    # Layer 6: Build profiles
    profiles, tag_scores_df = build_engineer_skill_profiles(
        df_skill, columns['engineer'], CONFIG
    )
    engineer_skills_df = create_output_dataframe(profiles, CONFIG['top_k_output'])
    
    # Layer 7: Build centroids
    engineer_centroids = build_engineer_centroids(df_skill, X_tfidf, columns['engineer'])
    
    # Layer 9: Save everything
    save_models(profiles, engineer_centroids, tfidf_tag)
    save_output(engineer_skills_df)
    
    # Display results
    print("\n=== Engineer Skills DataFrame ===")
    print(engineer_skills_df.head(10))
    print(f"\nTotal engineers: {len(engineer_skills_df)}")
    
    print("\n" + "=" * 80)
    print("PIPELINE COMPLETED SUCCESSFULLY")
    print("=" * 80)
    
    return engineer_skills_df, profiles, engineer_centroids, tfidf_tag

### EXECUTION

In [11]:
# Jalankan pipeline
engineer_skills_df, profiles, engineer_centroids, tfidf_tag = main()

# Lihat hasil
print("\n=== TOP 10 ENGINEERS ===")
print(engineer_skills_df.head(10))

# Lihat detail satu engineer
print("\n=== EXAMPLE: Skill Profile ===")
if len(engineer_skills_df) > 0:
    first_engineer = engineer_skills_df.iloc[0]['engineer']
    print(f"Engineer: {first_engineer}")
    print(f"Skills: {engineer_skills_df.iloc[0]['top_tags']}")

ENGINEER SKILL PROFILING PIPELINE
Dataset loaded: 24762 rows

=== Detected Columns ===
summary: Summary_x
judul: Judul Request_x
description: Description
status_x: Status_x
status_y: Status_y
engineer: Engineer
request_name: Request Name

=== Preprocessing Data ===
Applying text cleaning pipeline...
Rows included for skill profiling: 22338 / 24762
Final skill dataset: 22338 rows

=== TF-IDF Vectorization ===
TF-IDF vocab size: 3683
Extracting top tags per ticket...


Extracting tags:   0%|          | 0/22338 [00:00<?, ?it/s]


=== Building Engineer Skill Profiles with Scores ===
Engineer profiles created: 51 engineers

=== Building Engineer Centroids ===
Engineer centroids created: 51 engineers

=== Saving Models ===
Saved: engineer_profiles_tags.joblib
Saved: engineer_centroids_tfidf.joblib
Saved: engineer_skills_with_scores.csv

=== Engineer Skills DataFrame ===
                       engineer  \
0                    Aby Irawan   
1      Achmad Koesnadi Alamsyah   
2           Achmad Mahara Fauzy   
3             Ade Ilham Mustofa   
4              Adian Tampubolon   
5                Adilah Ahsanah   
6               Adithia Jovandy   
7        Aldy Kurniawan Syafawi   
8           Andhi Yudha Triawan   
9  Anggito Anju Hartawan Manalu   

                                            top_tags  
0  outq(0.727), data(0.715), adhoc(0.678), restor...  
1  vpn(0.714), access(0.648), akses(0.537), serve...  
2  server(0.730), machine(0.693), non(0.610), vir...  
3  data(0.735), maintenance(0.698), consolidated(